In [8]:
import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import pytorch_lightning as pl

import torchvision
from torchvision import datasets, models, transforms

from sklearn.metrics import f1_score

DATA_PATH = '../../datasets/crack-detection'

In [9]:
torch.cuda.empty_cache()

print(f'CUDA {torch.version.cuda}')
print(f'cuDNN {torch.backends.cudnn.version()}')

CUDA 10.2
cuDNN 7604


In [10]:
def get_datasets(data_path):
    image_folder = datasets.ImageFolder(data_path)

    train_size = int(0.5 * len(image_folder))
    valid_size = int(0.3 * len(image_folder))
    test_size  = len(image_folder) - (train_size + valid_size)
    train_ds, val_ds, test_ds = torch.utils.data.random_split(image_folder, [train_size, valid_size, test_size])

    print(f'There are {len(train_ds)} train., {len(val_ds)} valid. and {len(test_ds)} test samples.')

    train_ds.dataset.transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    val_ds.dataset.transform =  transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    test_ds.dataset.transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    return train_ds, val_ds, test_ds

In [11]:
def get_model(num_classes):
    model = models.vgg16(pretrained=True)
        
    for param in model.parameters():
        param.requires_grad = False
        
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

In [12]:
class SurfaceCrackDetectionModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = get_model(num_classes=2)
    
    def forward(self, x):
        x = self.model(x)
        return x

    def prepare_data(self):
        self.train_dataset, self.val_dataset, self.test_dataset = get_datasets(data_path=DATA_PATH)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=16)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=16)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset, batch_size=16)

    def configure_optimizers(self):
        optimizer = optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)
        scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
        return [optimizer], [scheduler]

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_hat = self.forward(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_hat = self.forward(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        tensorboard_logs = {'val_loss': loss}
        return {'val_loss': loss, 'log': tensorboard_logs}

    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        y_hat = self.forward(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        tensorboard_logs = {'test_loss': loss}
        return {'test_loss': loss, 'log': tensorboard_logs}

    def validation_epoch_end(self, outputs):
        val_loss_mean = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss_mean': val_loss_mean}
        return {'val_loss': val_loss_mean, 'log': tensorboard_logs}
    
    def test_epoch_end(self, outputs):
        test_loss_mean = torch.stack([x['test_loss'] for x in outputs]).mean()
        return {'test_loss': test_loss_mean}

In [13]:
model = SurfaceCrackDetectionModel()

tb_logger = pl.loggers.TensorBoardLogger('logs/')
trainer = pl.Trainer(
    gpus=1,
    max_epochs=5,
    logger=tb_logger,
    progress_bar_refresh_rate=0,
    train_percent_check=0.1,
    val_percent_check=0.1,
    test_percent_check=0.1
)

trainer.fit(model)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
There are 20000 train., 12000 valid. and 8000 test samples.

   | Name               | Type              | Params
-----------------------------------------------------
0  | model              | VGG               | 134 M 
1  | model.features     | Sequential        | 14 M  
2  | model.features.0   | Conv2d            | 1 K   
3  | model.features.1   | ReLU              | 0     
4  | model.features.2   | Conv2d            | 36 K  
5  | model.features.3   | ReLU              | 0     
6  | model.features.4   | MaxPool2d         | 0     
7  | model.features.5   | Conv2d            | 73 K  
8  | model.features.6   | ReLU              | 0     
9  | model.features.7   | Conv2d            | 147 K 
10 | model.features.8   | ReLU              | 0     
11 | model.features.9   | MaxPool2d         | 0     
12 | model.features.10  | Conv2d            | 295 K 
13 | model.features.11  | Re

1

In [14]:
trainer.test()

--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(0.0313, device='cuda:0')}
--------------------------------------------------------------------------------
